# Display of List of Available Widgets to Display 

## Overview 

The visualizer is planned to support three visualization modes:
1. Visualize updates to a standalone kvlang script
2. Visualize updates to an existing kivy widget
3. Visualize updates to an existing kivy application 

To allow the user to choose what to visualize, we should populate a listbox that displays each of the visualization options. The user will be able to select which item they would like to visualize. Switching between items will update the visualization. 

## Research

### How do you search a py file for class definitions?
We need to find all App and Widget descendants within each python file. We cannot expect all widget types to be a direct descendant of known kivy types. 

The `inspect` python module will return all of the base classes of a type, but this module requires live objects. We want to avoid executing the user's code, because this will require using their environment and we are trying to keep a clear separation between their environment and the kivy designer environment - plus actually executing their code can have some side effects. We should only do this execution during visualization, when they request it.

Unfortunately we cannot we use the `ast` module to find all objects within a python file that descend from the `App` and `widget` class. The reason is that the AST tree only stores the direct class parents, and does not provide any reference to where those parents are declared. To build a more complete hierarchy tree we would need to search through all the available source files and build the hierarchy ourselves. 


In [ ]:
import ast

# This code fails because ast does not store detailed information
# about ClassDef parents

def get_subclasses_of_widget(filename):
    subclasses = []
    with open(filename, "r") as f:
        tree = ast.parse(f.read())
        for node in ast.iter_child_nodes(tree):
            if isinstance(node, ast.ClassDef):
                if is_derived_from_widget(node):
                    subclasses.append(node.name)
    return subclasses

def is_derived_from_widget(node):
    for base in node.bases:
        if isinstance(base, ast.Name) and base.id == "Widget":
            return True
        elif is_derived_from_widget(base): #ERROR. Can't recursively traverse the tree this way. Base has no attr bases.
            return True
    return False

subclasses = get_subclasses_of_widget("C:\\Users\\joshu\\source\\repos\\kivydesigner\\kivydesigner\\uix\\kdfilechooser.py")
print(subclasses)


### What entries should the listbox contain?

(In this order)
1. Names of custom user defined applications
2. Names of custom user defined kvlang scripts
3. Names of custom user defined widgets
4. Names of kivy standard library widgets

In the future we should also populate the listbox with kivy garden widgets. 

### How should the entries be displayed?

A treeview would be appropriate here, since this will allow us to categorize the widgets into groups. 

Unfortunately, we probably won't be able to reuse some of the styling work we did on the kdfileviewer since the file viewer has a complex interplay of objects - but we can apply the same learnings here at the expense of some duplication. 

## General Solution Plan



1. Define a project search path for the user. We will default this to recursively search all of the paths in the file directory, but we will exclude a few key directories such as directories containing site-packages. This search path information will be provided as a project setting, to allow the user to customize it (in a future issue)
2. Parse each of the py files in the search path, and construct their AST. 
3. Use the AST to incrementally build a collection of inheritance trees
4. Search the inheritance trees to find all of the classes that descend from 'App' and 'Widget'. Doing this will require us to search for all kivy widget and application classes. We will need this list as a hard-coded value in our own code. 
5. Populate a listview with the list of available applications and widgets. 
    * Do not stylize the listview yet. Let's save this for a future issue when the visualizer is working
    * Don't forget to add kvlang files as well.
6. Use watchdog to listen for changes to the source files. Any time a change is made, refresh the inheritance by re-parsing the updated file. 
    * Also include a 'refresh' button. We will probably only listen for changes in files that are open in the visualizer.
7. (optional) Add decorators that can be used to force a widget to be visualized. One way to achieve this would be to add the 'Widget' or 'App' class whenever the decorator is detected.

This issue will create the class list widget, but will not implement the code needed to actually visualize the listed widgets.

## Implementation Questions

### How should `WidgetScanner` search a directory for widgets and apps?

We can search for widget and application classes without compiling the code if we construct inheritance trees from the project source code. The graphs will need to be built as class definitions are encountered, so a collection of disconnected inheritance trees need to be built.

We will need to use the implemented build functions with a search path, instead of naively building from a directory, to avoid parsing unnecessary files - such as the virtual environment source code files. 

In [ ]:
import ast 
from pathlib import Path
from dataclasses import dataclass

@dataclass
class ClassDefNode:
    name: str
    source_path: str
    parents: list[str]
    children: list[str]

class InheritanceGraphs:
    def __init__(self) -> None:
        self.nodes = dict()

    def add_classdef(self, source_path, classname, parent_classnames):
        '''
        Add a class definition and parents to the graph. 
        If the class already exists, update its parents.
        If the parents already exist, update their children.
        Ignore duplicate entries. Enforce that a class can only have one set of parents.
        '''
        class_node = self.nodes.get(classname)
        if class_node:
            if len(class_node.parents) == 0:
                class_node.parents = parent_classnames
            elif class_node.parents == parent_classnames:
                # Duplicate entry. Ignore and exit. 
                return
            else:
                raise Exception("Class already exists with different parents.")
        else:
            class_node = ClassDefNode(classname, source_path, parent_classnames, list())
            self.nodes[classname] = class_node

        for parent_classname in parent_classnames:
            parent_node = self.nodes.get(parent_classname)
            if parent_node:
                parent_node.children.append(class_node.name)
            else:
                parent_node = ClassDefNode(parent_classname, source_path, list(), [class_node.name])
                self.nodes[parent_classname] = parent_node

    def get_subclasses(self, classname):
        '''
        Return a list of all subclasses of the given class.
        '''
        class_node = self.nodes.get(classname)
        if class_node:
            subclasses = set(class_node.children)
            for child in class_node.children:
                subclasses.update(self.get_subclasses(child))
            return subclasses
        return set()

    def get_superclasses(self, classname):
        '''
        Return a list of all superclasses of the given class.
        '''
        class_node = self.nodes.get(classname)
        if class_node:
            subclasses = set(class_node.parents)
            for parent in class_node.parents:
                subclasses.update(self.get_subclasses(parent))
            return subclasses
        return set()

    def get_all_classes(self):
        return self.nodes.keys()

    def remove_class(self, classname):
        '''
        Remove the given class from the graph.
        '''
        class_node = self.nodes.get(classname)
        if class_node:
            for parent in class_node.parents:
                parent_node = self.nodes.get(parent)
                parent_node.children.remove(class_node)
            for child in class_node.children:
                child.parents.remove(class_node)
            del self.nodes[classname]
    
    def remove_class_and_subclasses(self, classname):
        '''
        Remove the given class and all its subclasses from the graph.
        '''
        self.remove_class(classname)
        for subclass in self.get_subclasses(classname):
            self.remove_class(subclass)


class InheritanceGraphsBuilder(ast.NodeVisitor):
    '''
    Build inheritance graphs from python source code,
    without executing any code by parsing the AST.
    '''

    def __init__(self) -> None:
        self.current_filepath = None
        self.graphs = InheritanceGraphs()

    def visit_ClassDef(self, node):
        parents = list()
        for base in node.bases:
            if isinstance(base, ast.Name):
                # Standard base class declaration
                parents.append(base.id)
            elif isinstance(base, ast.Attribute):
                # Parent of form module.Class
                parents.append(base.attr)
            elif isinstance(base, ast.Subscript):
                # Generic Parent of form Generic[T1,...,Tn]
                if isinstance(base.slice, ast.Name):
                    type_params = [base.slice.id]
                elif isinstance(base.slice, ast.Tuple):
                    type_params = [e.id for e in base.slice.elts]
                elif isinstance(base.slice, ast.Constant):
                    type_params = [base.slice.value]
                else:
                    raise Exception("Unknown subscript type: " + str(type(base.slice)))
                parents.append(base.value.id + "[" + ",".join(type_params) + "]")
            elif isinstance(base, ast.Call):
                # Call classdef bases not supported.
                # Our goals is to avoid executing any code, and
                # we can't determine the return value of a Call
                # node without execution.
                pass
            else:
                raise Exception("Unknown base type: " + str(type(base)))
                
        self.graphs.add_classdef(self.current_filepath, node.name, parents)

    def build(self, file_source, source_filepath=None):
        tree = ast.parse(file_source)
        self.current_filepath = source_filepath
        self.visit(tree)
        self.current_filepath = None
        return self.graphs

    def build_from_file(self, filepath):
        try:
            file_source = Path(filepath).read_text()
            return self.build(file_source, filepath)
        except:
            return None

    def build_from_directory(self, directory):
        for filepath in Path(directory).rglob('**/*.py'):
            self.build_from_file(filepath)

EX_FILE = "C:\\Users\\joshu\\source\\repos\\kivydesigner\\kivydesigner\\uix\\kdfilechooser.py"
EX_FILE1 = "C:\\Users\\joshu\\source\\repos\\kivydesigner\\kivydesigner\\uix\\iconbutton.py"
EX_FILE2 = "C:\\Users\\joshu\\source\\repos\\kivydesigner\\kivydesigner\\uix\\modalmsg.py"
EX_FILE3 = "C:\\Users\\joshu\\source\\repos\\kivydesigner\\kivydesigner\\uix\\resources.py"
EX_FILE4 = "C:\\Users\\joshu\\source\\repos\\kivydesigner\\kivydesigner\\uix\\toolbar.py"

builder = InheritanceGraphsBuilder()
builder.build_from_directory("C:\\Users\\joshu\\source\\repos\\kivydesigner")
apps = builder.graphs.get_subclasses("App")
widgets = builder.graphs.get_subclasses("Widget")
print(apps)
print("Widget subclasses: ")
print(widgets)

### How can we update our inheritance graphs when a file is out of date?

Each node of the inheritance stores a filepath. We can refresh these entries by doing the following:
1. Remove all nodes that contain that filepath
2. Parse the new file ast
3. `Build` the inheritance graph using the new file ast

This will remove all trace of the old declaration and replace it with the up-to-date declaration. This update to the inheritance graphs could dramatically change the subclasses of widget, so we will need to refresh the current application and widget lists. 